In [0]:
from IPython.display import clear_output

## Семинар 1 Индекс

## Intro

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
!pip3 install pymorphy2[fast]
clear_output()

In [0]:
%load_ext pycodestyle_magic

In [4]:
from google import colab
colab.files.upload()

Saving test.txt to test.txt


{'test.txt': b'Hello world'}

### работа с файлами и папками

In [0]:
import os

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [6]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному ти до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

/content/test.txt
test.txt
True


### os.listdir  
возвращает список файлов в данной директории

In [7]:
os.listdir(curr_dir)

['.config', 'test.txt', 'sample_data']

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [8]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

/content/test.txt
/content/.config/active_config
/content/.config/.last_update_check.json
/content/.config/gce
/content/.config/config_sentinel
/content/.config/.metricsUUID
/content/.config/configurations/config_default
/content/.config/logs/2019.08.27/16.17.17.407104.log
/content/.config/logs/2019.08.27/16.17.21.663235.log
/content/.config/logs/2019.08.27/16.17.07.367707.log
/content/.config/logs/2019.08.27/16.17.00.530950.log
/content/.config/logs/2019.08.27/16.17.22.686467.log
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/mnist_train_small.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_test.csv


> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [9]:
list(os.walk(curr_dir))

[('/content', ['.config', 'sample_data'], ['test.txt']),
 ('/content/.config',
  ['configurations', 'logs'],
  ['active_config',
   '.last_update_check.json',
   'gce',
   'config_sentinel',
   '.metricsUUID']),
 ('/content/.config/configurations', [], ['config_default']),
 ('/content/.config/logs', ['2019.08.27'], []),
 ('/content/.config/logs/2019.08.27',
  [],
  ['16.17.17.407104.log',
   '16.17.21.663235.log',
   '16.17.07.367707.log',
   '16.17.00.530950.log',
   '16.17.22.686467.log']),
 ('/content/sample_data',
  [],
  ['README.md',
   'anscombe.json',
   'mnist_train_small.csv',
   'mnist_test.csv',
   'california_housing_train.csv',
   'california_housing_test.csv'])]

### чтение файла 

In [0]:
filepath = 'test.txt'


# одним массивом  
with open(filepath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(filepath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(filepath, 'r') as f:   
    text = f.read().splitlines() 

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

In [0]:
!wget https://www.dropbox.com/s/yy50phca0ptp0jj/friends.zip?dl=0 -O friends.zip
clear_output()

In [0]:
!unzip friends.zip
clear_output()

In [0]:
!rm friends.zip
clear_output()

In [0]:
filepath = "friends"

In [0]:
import re
import sys
import pickle
from math import log
from pymorphy2 import MorphAnalyzer
from collections import Counter
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer

In [0]:
pattern = r"[а-яА-ЯёЁ]+\-?[а-яА-ЯёЁ]*"

m = MorphAnalyzer()
tokenizer = RegexpTokenizer(pattern)


def lemmatize(text):
    return ([m.parse(word)[0].normal_form
            for word in tokenizer.tokenize(text)])

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

###Result

In [0]:
class SearchEngine():
    def __init__(self, filepath):
        self.texts = dict()
        self.inv_index = defaultdict(list)
        self.preprocess(filepath)
        self.make_index()

    def preprocess(self, filepath):
        for root, dirs, files in os.walk(filepath):
            for name in files:
                with open(os.path.join(root, name), "r") as f:
                    self.texts[name[:-4]] = lemmatize(f.read())

    def make_index(self):
        for name in self.texts:
            for term in self.texts[name]:
                if name not in self.inv_index[term]:
                    self.inv_index[term].append(name)

    def tf_idf(self, term, document):
        tf = self.texts[document].count(term)
        idf = log(len(self.texts) / len(self.inv_index[term]))
        return tf * idf

    def search(self, query):
        result = defaultdict(float)
        for term in lemmatize(query):
            for document in self.inv_index[term]:
                result[document] += self.tf_idf(term, document)
        return [item[0] for item in
                sorted(result.items(), key=lambda kv: -kv[1])]

In [0]:
FriendsSearchEngine = SearchEngine(filepath)

In [21]:
# _check : в коллекции должно быть около 165 файлов
len(FriendsSearchEngine.texts)

165

In [22]:
for res in FriendsSearchEngine.search("""Да нечего рассказывать!
Он просто сотрудник!"""):
    print(res)

Friends - 1x01 - The One Where Monica Gets A Roommate.ru
Friends - 2x19 - The One Where Eddie Won't Go.ru
Friends - 5x12 - The One With Chandler's Work Laugh.ru
Friends - 7x03 - The One With Phoebe's Cookies.ru
Friends - 3x01 - The One With The Princess Leia Fantasy.ru
Friends - 4x20 - The One With The Wedding Dresses.ru
Friends - 5x07 - The One Where Ross Moves In.ru
Friends - 7x04 - The One With Rachel's Assistant.ru
Friends - 6x11 - The One With The Apothecary Table.ru
Friends - 2x23 - The One With The Chicken Pox.ru
Friends - 7x07 - The One With Ross's Library Book.ru
Friends - 1x16 - The One With Two Parts (1).ru
Friends - 2x11 - The One With The Lesbian Wedding.ru
Friends - 2x04 - The One With Phoebe's Husband.ru
Friends - 2x05 - The One With Five Steaks And An Eggplant.ru
Friends - 7x17 - The One With The Cheap Wedding Dress.ru
Friends - 6x24 - The One With The Proposal (1).ru
Friends - 3x10 - The One Where Rachel Quits.ru
Friends - 3x18 - The One With The Hypnosis Tape.ru
Frien

С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [23]:
max_freq = -sys.maxsize - 1
max_freq_word = None
for word in FriendsSearchEngine.inv_index:
    freq = 0
    for document in FriendsSearchEngine.inv_index[word]:
        freq += FriendsSearchEngine.texts[document].count(word)
    if freq > max_freq:
        max_freq = freq
        max_freq_word = word
print(f"Самое частотное слово: \"{max_freq_word}\".")

Самое частотное слово: "я".


In [24]:
min_freq = sys.maxsize
min_freq_word = None
for word in FriendsSearchEngine.inv_index:
    freq = 0
    for document in FriendsSearchEngine.inv_index[word]:
        freq += FriendsSearchEngine.texts[document].count(word)
    if freq < min_freq:
        min_freq = freq
        min_freq_word = word
print(f"Самое редкое слово: \"{min_freq_word}\".")

Самое редкое слово: "джуана".


In [25]:
universal_words = []
for word in FriendsSearchEngine.inv_index:
    if len(FriendsSearchEngine.inv_index[word]) == 165:
        universal_words.append(word)
universal_words.sort()
print(f"Слова, которые есть во всех документах коллекции:\n{universal_words}")

Слова, которые есть во всех документах коллекции:
['а', 'быть', 'в', 'весь', 'да', 'думать', 'если', 'ещё', 'знать', 'и', 'как', 'мой', 'мочь', 'мы', 'на', 'не', 'нет', 'но', 'ну', 'о', 'он', 'она', 'просто', 'с', 'сказать', 'так', 'такой', 'тот', 'ты', 'у', 'хотеть', 'что', 'это', 'этот', 'я']


In [26]:
count = Counter([re.search(r"(\d)x\d\d", document).group(1) for document
                 in FriendsSearchEngine.inv_index["чендлер"]])
print(f"Самый популярный сезон Чендлера: {count.most_common(1)[0][0]}")

Самый популярный сезон Чендлера: 6


In [27]:
count = Counter([re.search(r"(\d)x\d\d", document).group(1) for document
                 in FriendsSearchEngine.inv_index["моника"]])
print(f"Самый популярный сезон Моники: {count.most_common(1)[0][0]}")

Самый популярный сезон Моники: 2


In [28]:
def popularity(name):
    name = name.lower()
    freq = 0
    for document in FriendsSearchEngine.inv_index[name]:
        freq += FriendsSearchEngine.texts[document].count(name)
    return freq


names = ["Рэйчел", "Моника", "Фиби", "Джоуи", "Чендлер", "Росс"]
chars = {name: popularity(name) for name in names}
print(f"Самый популярный герой: {max(chars, key=lambda key: chars[key])}.")

Самый популярный герой: Росс.


In [0]:
with open("FriendsSearch.pkl", "wb") as file:
    pickle.dump(FriendsSearchEngine, file)

In [0]:
colab.files.download("FriendsSearch.pkl")